<a href="https://colab.research.google.com/github/takhanhvy/-Sentiment-Analysis-with-RNN-LSTM/blob/main/Sentiment_Analysis_with_RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis for mental health with RNN & NLP

**Réalisé par : TA Khanh Vy**  
Dans le cadre du cours "Natural Language Processing" du M1 Mastère Data Engineering (Efrei) dirigé par Mme. Sarah Malaeb

## Contexte

Ce notebook vise à mener une analyse sentimentale à partir des posts sur les réseaux sociaux dans le but d'identifier automatiquement et catégoriser les sentiments exprimés dans les contenus publiés sur les réseaux sociaux.

L’objectif est de développer un modèle de Deep Learning basé sur un réseau neuronal récurrent (RNN/LSTM) capable de prédire le sentiment à partir des textes dans les postes publiés.

Ce modèle doit classer chaque texte dans l’une des " catégories suivantes :
- Positive  
- Negative  
- Neutral  

 **Jeu de données utilisé :**

Le dataset provient de Kaggle : [Sentiment Analysis](https://www.kaggle.com/datasets/mdismielhossenabir/sentiment-analysis).  

Les colonnes principales sont :
- `text` : le texte de la publication
- `sentiment` : l’étiquette de sentiment associée

## Importation des bibliothèques


In [ ]:
# nettoyage et prétraitement des données
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk

# téléchargement des stopwords et stemmatisation
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# entrainement du modèle RNN/LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# évaluation de la performance du modèle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

# création des combinaisons d'hyperparamètres
from itertools import product

## Chargement des données

In [ ]:
# charger le fichier csv
from google.colab import files
uploaded = files.upload()

In [ ]:
# lire des données
df = pd.read_csv("sentiment_analysis.csv")
df.head()

## Analyse exploratoire

In [ ]:
# distribution des sentiments
sentiments = df["sentiment"].value_counts()

plt.figure()
sentiments.plot(kind="bar")
plt.title("Répartition des classes")
plt.xlabel("Classe")
plt.ylabel("Nombre d'exemples")
plt.show()

In [ ]:
# distribution de la longeur du texte
text_length = df['text'].apply(len)
plt.hist(text_length)
plt.title("Text Length Distribution")
plt.xlabel("Text length")
plt.ylabel("Count")
plt.show()

## Nettoyage et prétraitement des données

In [ ]:
# vérifier les valeurs nulles
df.isnull().sum()

In [ ]:
# nettoyer les données

# initializer lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # conversion en miniscule
    text = text.lower()
    # suppression des charactères spéciaux, des chiffres et de la punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # suppression des urls
    text = re.sub(r"http\S+|www\S+", "", text)
    # tokenization simple (pour nettoyage)
    words = text.split()
    # supprimer stopwords and appliquer lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # appliquer le stemming
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

In [ ]:
# appliquer le nettoyage
df['processed_text'] = df['text'].apply(preprocess_text)

## Division du jeu de données en train et test

In [ ]:
# features et labels
X = df['processed_text']
y = df['sentiment']

In [ ]:
# encoder les labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# division en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

## Entrainement du modèle RNN

In [ ]:
# tokenization pour RNN et LSTM
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
# tokenizer les textes en vecteurs numériques
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Padding pour avoir la même longeur pour chaque texte
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [ ]:
# convertir les labels en catégories
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
# contruire le modèle RNN simple
rnn_model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    SimpleRNN(64, return_sequences=False),
    Dropout(0.2),
    Dense(3, activation='sigmoid')
])

In [ ]:
# compiler avec Adam
rnn_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [ ]:
# Entraîner le modèle
history = rnn_model.fit(X_train_pad, y_train_cat, epochs=10, batch_size=32,   validation_data=(X_test_pad, y_test_cat), verbose=1)

## Test et évaluation de la performance du modèle RNN

In [ ]:
# prédictions sur le test
y_pred_rnn = rnn_model.predict(X_test_pad)

In [ ]:
# convertir les probabilités en classes prédites
y_pred_rnn_classes = np.argmax(y_pred_rnn, axis=1)

# Évaluation : accuracy, f1 score, recall_score
accuracy = accuracy_score(y_test, y_pred_rnn_classes)
f1 = f1_score(y_test, y_pred_rnn_classes, average='weighted')

print('RNN Model Performance:')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')
print('Classification Report:')
print(classification_report(y_test, y_pred_rnn_classes, target_names=label_encoder.classes_))

In [ ]:
# matrice de confusion
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred_rnn_classes), annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('RNN Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

Le modèle RNN obtient environ 66% de bonnes prédictions, ce qui est correct pour un RNN simple sur un dataset textuel brut. Le score F1 proche de l’accuracy indique aussi que la performance est assez équilibrée entre les classes.

Si on fait une analyse par classe :  
- `negative` : Le modèle a du mal à identifier les messages négatifs. Il confond souvent cette classe avec les neutres. Il y a 11 prédictions correctes pour "negative" et 12 erreurs vers “neutral”. Cela confirme la faiblesse observée dans le recall négatif (0.41).
- `neutral` : C’est la classe la mieux reconnue, avec 34 prédictions correcte qui est un très bon score, et quelques erreurs (4 vers énegative" et 2 vers "positive". Le modèle prédit souvent neutral, ce qui explique son recall très élevé (85%). Il a tendance à prédire “neutre” dès qu'il n’est pas certain et cette classe est possiblement sur-apprise par le modèle.
- `positive` : La classe positive fonctionne bien en précision (78%) avec 21 bonnes prédictions, ce qui signifie que lorsque le modèle prédit positif, il se trompe peu.

Globalement, le modèle RNN pour ce cas d'usage semble biaisé vers la classe "neutral" (comme souvent avec des RNN peu profonds). Cela pourra indiquer que le modèle préfère jouer "safe" et classer les textes dans la catégorie la plus “facile”

In [ ]:
# visualiser les historique d'entraînement : courbes accuracy/loss RNN
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('RNN Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('RNN Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

- **Courbe d’Accuracy :**  
L’accuracy d’entraînement monte très rapidement jusqu’à 100%, ce qui signifie que le modèle mémorise très facilement les exemples du training set. En revanche, l’accuracy de validation plafonne autour de 0.70 et évolue très lentement. L’écart entre les deux courbes augmente de façon continue à partir des premières epochs.  
=> Cela permet de révéler un surapprentissage (overfitting) clair et précoce.
Le modèle apprend à reconnaître parfaitement les données du training set, mais ne généralise pas bien sur de nouvelles données.


- **Courbe de Loss :**  
La train loss diminue fortement de 0.22 -> 0.02. La validation loss diminue aussi mais beaucoup moins et avec un plateau (≈0.15). La validation loss reste systématiquement plus haute que la train loss.  
=> Encore une fois, on peut observer un phénomène de surapprentissage. La différence entre train loss et validation loss indique que le modèle apprend trop bien les données d’entraînement, mais ne capture pas de manière robuste les motifs généralisables.

## Variantes LSTM

In [ ]:
# construire le modèle LSTM
lstm_model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_words),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

# compiler
lstm_model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

# entraîner
history_lstm = lstm_model.fit(X_train_pad, y_train_cat, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test_cat), verbose=1)

In [ ]:
# évaluer la performance du modèle LSTM

y_pred_lstm = lstm_model.predict(X_test_pad)
y_pred_lstm_classes = np.argmax(y_pred_lstm, axis=1)

accuracy = accuracy_score(y_test, y_pred_lstm_classes )
f1 = f1_score(y_test, y_pred_lstm_classes , average='weighted')

print('LSTM Model Performance:')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')
print('Classification Report:')
print(classification_report(y_test, y_pred_lstm_classes, target_names=label_encoder.classes_))

In [ ]:
# matrice de confusion
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred_lstm_classes), annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('LSTM Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

Le modèle LSTM améliore nettement les performances par rapport au RNN simple. Avec une accuracy de 74 % et un F1-score de 0.73, il parvient à mieux distinguer les trois classes de sentiment. Les scores de précision et de rappel sont également plus équilibrés : le LSTM identifie mieux les messages négatifs et positifs, tout en conservant une excellente performance sur la classe neutre. Globalement, il généralise mieux aux données de test et réduit les biais observés précédemment.

La matrice de confusion confirme cette amélioration : les erreurs les plus fréquentes du RNN (notamment la confusion "negative" -> "neutral") sont nettement réduites. Le LSTM classe correctement la majorité des messages dans chaque catégorie, avec une répartition des erreurs plus homogène et moins systématique. Cette meilleure capacité de séparation montre que le LSTM capture mieux le contexte des phrases, ce qui le rend plus efficace pour l'analyse de sentiments.

In [ ]:
# visualiser les historique d'entraînement : courbes accuracy/loss LSTM
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['accuracy'], label='Train Accuracy')
plt.plot(history_lstm.history['val_accuracy'], label='Validation Accuracy')
plt.title('LSTM Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_lstm.history['loss'], label='Train Loss')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss')
plt.title('LSTM Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

Les courbes d’accuracy montrent une progression régulière de l’apprentissage : l’accuracy d’entraînement augmente de façon stable jusqu’à dépasser 95 %, tandis que l’accuracy de validation progresse également mais de manière plus modérée, se stabilisant autour de 73–75 %. Cela indique que le modèle apprend efficacement les motifs du dataset tout en parvenant à généraliser correctement, même si un léger écart persiste entre les deux courbes. Par rapport au RNN simple, la validation accuracy est plus élevée et surtout plus stable, ce qui confirme une meilleure capacité du LSTM à capturer la structure séquentielle du texte.

Les courbes de loss confirment cette observation : la training loss diminue fortement, ce qui montre un apprentissage performant, tandis que la validation loss suit aussi une tendance descendante mais reste plus élevée et légèrement fluctuante. Cela suggère une petite dose d’overfitting, mais nettement moins prononcée que pour le RNN. Globalement, ces courbes indiquent que le LSTM converge bien et généralise mieux, tout en profitant d’une architecture plus robuste pour traiter des données textuelles complexes.

## Optimisation d’hyperparamètres

On teste plusieurs combinaisons :
- Nombre d’unités (taille cachée)
- Nombre de couches
- Taux d’apprentissage
- Taille du batch
- Taux de dropout
- Longueur de séquence / fenêtre d’entrée

In [ ]:
# =========================================================
# 1) Fonction qui construit un modèle selon hyperparamètres
# =========================================================
def build_model(rnn_type="LSTM", units=64, n_layers=1, dropout=0.2, lr=1e-3, max_len=100):
    """
    Construit un modèle LSTM configurable.
    - rnn_type : "LSTM"
    - units : nb d'unités par couche récurrente
    - n_layers : profondeur (1 = simple, >1 = empilé)
    - dropout : taux dropout
    - lr : learning rate Adam
    - max_len : longueur de séquence (important pour input_length)
    """
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))

    # Ajout des couches récurrentes
    for i in range(n_layers):
        return_seq = (i < n_layers - 1)  # True sauf dernière couche
        model.add(LSTM(units, return_sequences=return_seq))
        model.add(Dropout(dropout))

    # Couche de sortie : 3 classes -> softmax
    model.add(Dense(3, activation='softmax'))

    # compiler avec optimiseur 'adam' et la fonction de perte calculant l’erreur quadratique moyenne (MSE)
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss="mse",
        metrics=["accuracy"]
    )
    return model


# =========================================================
# 2) Définition de l’espace de recherche
# =========================================================
param_grid = {
    "rnn_type": ["LSTM"],
    "units": [32, 64, 128], # Nombre d’unités
    "n_layers": [1, 2],     # Nombre de couches
    "dropout": [0.2, 0.4],  # Taux de dropout
    "lr": [1e-3, 5e-4],     # Taux d'apprentissage
    "batch_size": [32, 64], # Taille du batch
    "max_len": [80, 120]    # Longueur de séquence
}

grid_list = list(product(
    param_grid["rnn_type"],
    param_grid["units"],
    param_grid["n_layers"],
    param_grid["dropout"],
    param_grid["lr"],
    param_grid["batch_size"],
    param_grid["max_len"]
))

print("Nombre total de combinaisons testées :", len(grid_list))


# =========================================================
# 3) Boucle d’entraînement / évaluation
# =========================================================
results = []

for idx, (rnn_type, units, n_layers, dropout, lr, batch_size, max_len_test) in enumerate(grid_list, 1):

    print(f"\n--- Test {idx}/{len(grid_list)} | {rnn_type}, units={units}, layers={n_layers}, "
          f"dropout={dropout}, lr={lr}, batch={batch_size}, max_len={max_len_test} ---")

    # Re-padding selon la longueur de séquence testée
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq  = tokenizer.texts_to_sequences(X_test)
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_len_test)
    X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len_test)

    # Build + train
    model = build_model(rnn_type=rnn_type, units=units, n_layers=n_layers,
                        dropout=dropout, lr=lr, max_len=max_len_test)

    history = model.fit(
        X_train_pad, y_train_cat,
        epochs=5,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=0
    )

    # Prédiction + métriques
    y_pred = model.predict(X_test_pad, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test_cat, axis=1)

    acc = accuracy_score(y_true_classes, y_pred_classes)
    f1  = f1_score(y_true_classes, y_pred_classes, average="weighted")
    test_loss, test_acc = model.evaluate(X_test_pad, y_test_cat, verbose=0)

    results.append({
        "rnn_type": rnn_type,
        "units": units,
        "n_layers": n_layers,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "max_len": max_len_test,
        "test_loss": test_loss,
        "test_acc": test_acc,
        "f1_score": f1
    })



In [ ]:
# =========================================================
# 4) Résultats sous forme de tableau trié
# =========================================================
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="f1_score", ascending=False)

print("\nTop 10 meilleures combinaisons :")
display(results_df.head(10))


# =========================================================
# 5) Visualisation simple : F1 en fonction des configs
# =========================================================
plt.figure(figsize=(10,5))
plt.plot(results_df["f1_score"].values, marker="o")
plt.title("F1-score des combinaisons testées (triées)")
plt.xlabel("Combinaisons (triées)")
plt.ylabel("F1-score (weighted)")
plt.grid(True)
plt.show()

Les résultats du tuning montrent que les meilleures performances sont obtenues avec des architectures LSTM relativement simples mais suffisamment expressives. Les configurations les plus efficaces utilisent généralement 128 unités, 1 seule couche, un dropout faible (0.2), un learning rate standard (0.001) et une taille de batch de 32. Ces combinaisons atteignent un F1-score maximal d’environ 0.61, ce qui reste inférieur au LSTM initial (≈0.73), confirmant que le modèle de base était mieux calibré pour ce dataset. On observe également que la longueur de séquence optimale se situe entre 80 et 120 tokens, ce qui montre que des séquences trop longues n’apportent pas de gain significatif.

La courbe des F1-scores triés illustre un déclin progressif et stable, typique des grilles d’hyperparamètres où quelques configurations seulement offrent de bons compromis. La majorité des combinaisons obtiennent des scores entre 0.35 et 0.50, ce qui indique que des hyperparamètres mal adaptés dégradent rapidement la capacité du modèle à généraliser. Globalement, cette exploration confirme que le choix des hyperparamètres influence fortement la performance, mais montre également que, dans ce cas précis, les configurations testées n’ont pas surpassé le modèle LSTM initial, confirmant que celui-ci représentait déjà un bon équilibre entre complexité et généralisation.